# Fine-Tuning LLMs

In this exercise, you will fine-tune the [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model for enhanced dialogue summarization. You will first explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter-Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

## 1. Set up Dependencies and Load Dataset and LLM

In [1]:
!pip install datasets evaluate rouge_score peft -q

In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset

In [3]:
dataset = load_dataset('knkarthick/dialogsum')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load the pre-trained [Flan-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of Flan-T5. Setting `torch_dtype=torch.bfloat16` specifies the data type to be used by this model, which can reduce GPU memory usage since `bfloat16` uses half as much memory per number compared to `float32`, the default precision for most models.

In [5]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 2. Test the Model with Zero-Shot Inferencing

Test the model with zero-shot inference.

In [35]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION:\n{original_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

You can see that the model struggles to summarize the dialogue compared to the baseline summary, and simply repeats the first sentence from the dialogue.

## 3. Perform Full Fine-Tuning

### 3.1 Preprocess the Dataset

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Summarize the following conversation.`, and to the start of the summary with `Summary:` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.
Alice: This is her part of the conversation.
Bob: This is his part of the conversation.    
Summary:
```

Training response (summary):
```
Both Alice and Bob participated in the conversation.
```

**Exercise**: Write a function to tokenize a batch of examples from the dialogue dataset. The function should concatentate the dialogues with the predefined prompt, tokenize them along with their summaries, and define the tokenized summaries as the labels.

In [6]:
def tokenize(examples):
    ### WRITE YOUR CODE HERE
    prefix = "Summarize the following conversation.\n"
    postfix = "\nSummary:"
    inputs = [prefix + doc + postfix for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### 3.2 Fine-Tune the Model

**Exercise**: Utilize the Hugging Face Trainer API for training the model on the preprocessed dataset. Define the training arguments, a data collator, and create a `Seq2SeqTrainer` instance. Train the model for one epoch.

In [8]:
# Training args
batch_size = 16
num_train_epochs = 1

args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/PA5 Assignment",
    do_train = True,
    do_eval = True,
    evaluation_strategy="steps",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=16,
    weight_decay=0.01,
    log_level="debug",
    logging_strategy="steps",
    num_train_epochs=num_train_epochs,
    seed=1,
    bf16=True,
    bf16_full_eval=True,
    eval_steps=10,
    logging_steps=10,
    save_strategy='steps',
    save_steps = 0.2,
    save_total_limit=3,
)


In [9]:
# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [10]:
# Seq2SeqTrainer instance
trainer = Seq2SeqTrainer(
    model=original_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Using auto half precision backend


Training a fully fine-tuned version of the model should take about 10 minutes on a Google Colab GPU machine.

In [11]:
trainer.train()

Currently training with a batch size of: 16
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, summary, dialogue, topic. If id, summary, dialogue, topic are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 12,460
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 48
  Number of trainable parameters = 247,577,856


Step,Training Loss,Validation Loss
10,1.559600,1.171397
20,1.270900,1.111279
30,1.221300,1.082012
40,1.199900,1.070874


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, summary, dialogue, topic. If id, summary, dialogue, topic are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10
Configuration saved in /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10/config.json
Configuration saved in /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10/generation_config.json
Model weights saved in /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10/model.safetensors
tokenizer config file saved in /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/PA5 Assignment/tmp-checkpoint-10/special_tokens_map.json
Copy vocab file to /content/drive/MyDri

TrainOutput(global_step=48, training_loss=1.2919357220331829, metrics={'train_runtime': 184.2385, 'train_samples_per_second': 67.63, 'train_steps_per_second': 0.261, 'total_flos': 7828445634527232.0, 'train_loss': 1.2919357220331829, 'epoch': 0.99})

Save the model to a local folder:

In [12]:
model_path = './flan-t5-base-dialogsum-checkpoint'
#model_path = '/content/drive/MyDrive/PA5 (Fine-Tuning LLMs) 1/flan-t5-base-dialogsum-checkpoint-1'

original_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in ./flan-t5-base-dialogsum-checkpoint/config.json
Configuration saved in ./flan-t5-base-dialogsum-checkpoint/generation_config.json
Model weights saved in ./flan-t5-base-dialogsum-checkpoint/model.safetensors
tokenizer config file saved in ./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json
Special tokens file saved in ./flan-t5-base-dialogsum-checkpoint/special_tokens_map.json
Copy vocab file to ./flan-t5-base-dialogsum-checkpoint/spiece.model


('./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json',
 './flan-t5-base-dialogsum-checkpoint/special_tokens_map.json',
 './flan-t5-base-dialogsum-checkpoint/spiece.model',
 './flan-t5-base-dialogsum-checkpoint/added_tokens.json',
 './flan-t5-base-dialogsum-checkpoint/tokenizer.json')

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

In [13]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-checkpoint',
                                                       torch_dtype=torch.bfloat16)
# instruct_model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/PA5 (Fine-Tuning LLMs)/checkpoint-80',
#                                                        torch_dtype=torch.bfloat16)

loading configuration file ./flan-t5-base-dialogsum-checkpoint/config.json
Model config T5Config {
  "_name_or_path": "./flan-t5-base-dialogsum-checkpoint",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,


Reload the original Flan-T5-base model:

In [14]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 2

### 3.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Section 2, using the original model and the fully fine-tuned model.

In [37]:
### WRITE YOUR CODE HERE
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
original_model_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(original_model_output, skip_special_tokens=True)

instruct_model_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(instruct_model_output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION:\n{original_model_summary}\n')
print(dash_line)
print(f'INSTRUCT MODEL GENERATION:\n{instruct_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

The fine-tuned model is able to create a much better summary of the dialogue compared to the original model.

### 3.4 Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [16]:
rouge = evaluate.load('rouge')

**Exercise**: Generate the outputs for a sample of the test set with the fine-tuned model (use only the first 10 dialogues and summaries to save time).

In [17]:
### WRITE YOUR CODE HERE
human_baseline_summaries = []
original_model_summaries = []
instruct_model_summaries = []

for idx in range(10):
  dialogue = dataset['test'][idx]['dialogue']
  summary = dataset['test'][idx]['summary']
  human_baseline_summaries.append(summary)

  prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
  inputs = tokenizer(prompt, return_tensors='pt')
  original_model_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
  original_model_summary = tokenizer.decode(original_model_output, skip_special_tokens=True)

  instruct_model_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
  instruct_model_summary = tokenizer.decode(instruct_model_output, skip_special_tokens=True)

  original_model_summaries.append(original_model_summary)
  instruct_model_summaries.append(instruct_model_summary)


Evaluate the models computing ROUGE metrics:

In [18]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)]
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)]
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.22754314914198637, 'rouge2': 0.09982267689684571, 'rougeL': 0.2116144901610018, 'rougeLsum': 0.21135161167428607}
INSTRUCT MODEL:
{'rouge1': 0.30749690080369374, 'rouge2': 0.07752467850387067, 'rougeL': 0.23535596814257592, 'rougeLsum': 0.23560446397803886}


The results show substantial improvement in all ROUGE metrics:

In [19]:
print("Absolute percentage improvement of the instruct model over the original model:")

for key in instruct_model_results:
    improvement = instruct_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the instruct model over the original model:
rouge1: 8.00%
rouge2: -2.23%
rougeL: 2.37%
rougeLsum: 2.43%


## 4. Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** instead of "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning, with comparable evaluation results as you will see soon.

One of the most popular PEFT methods is **Low-Rank Adaptation (LoRA)**, which  introduces low-rank matrices to adapt the LLM with minimal additional parameters. In most cases, when someone says PEFT, they typically mean LoRA.  After fine-tuning for a specific task with LoRA, the result is that the original LLM remains unchanged and a newly-trained "LoRA adapter" emerges. This LoRA adapter is much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

At inference time, the LoRA adapter is reunited and combined with its original LLM to serve the inference request. The benefit is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 4.1 Setup the LoRA model for Fine-Tuning

You first need to define the configuration of the LoRA model. Have a look at the configuration below. The key configuration element to adjust is the rank (`r`) of the adapter, which influences its capacity and complexity. Experiment with various ranks, such as 8, 16, or 32, and see how they affect the results.

In [20]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)

Add LoRA adapter layers/parameters to the original LLM to be trained:

In [21]:
peft_model = get_peft_model(original_model, lora_config)

The number of trainable model parameters in the LoRA model is:

In [22]:
peft_model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4092820552029972


### 4.2 Train the LoRA Adapter

**Exercise**: Define training arguments and create a `Seq2SeqTrainer` instance for the LoRA model. Use a higher learning rate than full fine-tuning (e.g., `1e-3`).

In [23]:
### WRITE YOUR CODE HERE

peft_training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/PA5 Assignment/PEFT",
    do_train = True,
    do_eval = True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=16,
    learning_rate=1e-3,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    logging_strategy="steps",
    seed=42,
    bf16=True,
    bf16_full_eval=True,
    eval_steps=10,
    save_strategy='steps',
    save_steps = 0.2,
    logging_steps = 10,
    save_total_limit=3
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
peft_model.config.use_cache = False

peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    args=peft_training_args,
    data_collator=data_collator,
    tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Using auto half precision backend


Train the PEFT adapter. Training should take about 6 minutes on a Google Colab GPU machine.

In [25]:
peft_trainer.train()

Currently training with a batch size of: 16
The following columns in the training set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: id, summary, dialogue, topic. If id, summary, dialogue, topic are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 12,460
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 48
  Number of trainable parameters = 3,538,944


Step,Training Loss,Validation Loss
10,1.604200,1.239341
20,1.358600,1.172541
30,1.306200,1.148917
40,1.275700,1.141850


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: id, summary, dialogue, topic. If id, summary, dialogue, topic are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/PA5 Assignment/PEFT/tmp-checkpoint-10
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act":

TrainOutput(global_step=48, training_loss=1.3687174320220947, metrics={'train_runtime': 181.1582, 'train_samples_per_second': 68.78, 'train_steps_per_second': 0.265, 'total_flos': 8053188106862592.0, 'train_loss': 1.3687174320220947, 'epoch': 0.99})

Save the model to a local folder:

In [26]:
peft_model.save_pretrained('./flan-t5-base-dialogsum-lora')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repe

Load the PEFT model:

In [27]:
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer

peft_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 2

Reload the original Flan-T5-base model:

In [28]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 2

### 4.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Sections 2 and 3, using the original model, the fully fine-tuned model and the PEFT model.

In [30]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')

output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

fine_tuned_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
fine_tuned_model_summary = tokenizer.decode(fine_tuned_output, skip_special_tokens=True)

peft_output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
peft_model_summary = tokenizer.decode(peft_output, skip_special_tokens=True)

print(dash_line)
print(f'DIALOGUE:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'ORIGINAL MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')
print(dash_line)
print(f'FINETUNED MODEL GENERATION:\n{fine_tuned_model_summary}\n')
print(dash_line)
print(f'PEFT MODEL GENERATION:\n{peft_model_summary}\n')

----------------------------------------------------------------------------------------------------
DIALOGUE:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-----------------------------------------------------------

### 4.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Exercise**: Generate the outputs for a sample of the test set with the PEFT model (use only the first 10 dialogues and summaries to save time).

In [31]:
### WRITE YOUR CODE HERE
human_baseline_summaries = []
original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx in range(10):
  dialogue = dataset['test'][idx]['dialogue']
  summary = dataset['test'][idx]['summary']
  human_baseline_summaries.append(summary)

  prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
  inputs = tokenizer(prompt, return_tensors='pt')
  original_model_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
  original_model_summary = tokenizer.decode(original_model_output, skip_special_tokens=True)

  instruct_model_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
  instruct_model_summary = tokenizer.decode(instruct_model_output, skip_special_tokens=True)

  peft_model_output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
  peft_model_summary = tokenizer.decode(peft_model_output, skip_special_tokens=True)

  original_model_summaries.append(original_model_summary)
  instruct_model_summaries.append(instruct_model_summary)
  peft_model_summaries.append(peft_model_summary)


Compute ROUGE score for this subset of the data.

In [32]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.22754314914198637, 'rouge2': 0.09982267689684571, 'rougeL': 0.2116144901610018, 'rougeLsum': 0.21135161167428607}
INSTRUCT MODEL:
{'rouge1': 0.30749690080369374, 'rouge2': 0.07752467850387067, 'rougeL': 0.23535596814257592, 'rougeLsum': 0.23560446397803886}
PEFT MODEL:
{'rouge1': 0.39535973659470647, 'rouge2': 0.13177960944978256, 'rougeL': 0.30808115600837016, 'rougeLsum': 0.30734436391762876}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Calculate the improvement of PEFT over the original model:

In [33]:
print("Absolute percentage improvement of the PEFT model over the original model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the original model:
rouge1: 16.78%
rouge2: 3.20%
rougeL: 9.65%
rougeLsum: 9.60%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [34]:
print("Absolute percentage improvement of the PEFT model over the instruct model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - instruct_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the instruct model:
rouge1: 8.79%
rouge2: 5.43%
rougeL: 7.27%
rougeLsum: 7.17%


You can see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources.